In [1]:
from libraries import *
from parameters import *

In [2]:
os.getcwd()
os.chdir(projectDir)

We will assess the effect of the perturbations on the set of genes that are expressed in at least 5% of the whole cell population.

In [3]:
adata = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_05242020.h5ad')


In [5]:
adata.X.shape

(341664, 13811)

In [71]:
# adata.uns['feature_Control_barcode_names'] = [ x for x in adata.uns['feature_barcode_names'] if (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET"))]
# koGenes = pd.Series(list(set([x.split("_")[0] for x in adata.uns['feature_barcode_names']])))
# koGenes.to_csv("KO_genes.csv", index=False)
# adataControl = adata[adata.obs.loc[:,adata.uns['feature_Control_barcode_names']].sum(axis=1) == 1].copy()
# koGenes = pd.read_csv('KO_genes.csv')
# koGenes.columns = ["KOGenes"]
# koGenes = pd.Series(koGenes.KOGenes)
# koGenes = koGenes[koGenes != "NO"]
# koGenes = koGenes[koGenes != "ONE"]

# koGenes[~koGenes.isin(adataControl.var_names)]

In [6]:
sc.pp.filter_genes(adata, min_cells=20000)
sc.pp.filter_cells(adata, min_genes=800)


filtered out 7126 genes that are detected in less than 20000 cells
filtered out 13007 cells that have less than 800 genes expressed


In [7]:
adata.X.shape

(328657, 6685)

In [8]:
fBarMat = adata.obs[adata.uns['feature_barcode_names']]

In [9]:
badFBarMat = fBarMat.loc[:,fBarMat.sum(axis=0) < 20]
badGuides = badFBarMat.columns.to_list()
badGDF = pd.DataFrame(data={'guides':badGuides})
badGDF['targetGene'] = badGDF['guides'].str.split("_", n=1, expand=True)[0]
#badGDF = badGDF[-badGDF['targetGene'].isin(['NO', 'ONE'])]

In [10]:
badControlGuides = pd.read_csv('OutlierControlGuides.csv')
badControlGuides['targetGene'] = badControlGuides['OutlierGuides'].str.split("_", n=1, expand=True)[0]
badControlGuides = badControlGuides.rename(columns={"OutlierGuides": "guides"})
badGDF = pd.concat([badGDF, badControlGuides])
badGDF = badGDF.sort_values('guides')

In [11]:
badGDF

guides targetGene
31     Aamp_3       Aamp
119   Abtb1_3      Abtb1
174   Acaca_3      Acaca
58      Ahr_3        Ahr
51     Aire_2       Aire
..        ...        ...
63    Zbtb6_2      Zbtb6
124  Zbtb8a_2     Zbtb8a
32    Znrf1_1      Znrf1
33    Znrf1_2      Znrf1
68    Znrf3_3      Znrf3

[221 rows x 2 columns]

In [12]:
badGDF.targetGene.value_counts()[0:20]

ONE       24
NO        11
Gnb4       3
Mdm2       3
Cdc20      3
Npm1       2
Notch1     2
Tle6       2
Plk1       2
Klhl2      2
Gm3701     2
Rad18      2
Znrf1      2
Rnf207     1
Rnf20      1
Rnf222     1
Rnf40      1
Rnf44      1
Aamp       1
Rnft1      1
Name: targetGene, dtype: int64

In [13]:
clusterSelectedBarcodes = adata.uns['feature_barcode_names']
clusterSelectedBarcodes = clusterSelectedBarcodes[~pd.Series(clusterSelectedBarcodes).isin(badGDF.guides.to_list())]
adata.uns['feature_barcode_names_filtered'] = clusterSelectedBarcodes


In [14]:
fBarMat = adata.obs[adata.uns['feature_barcode_names_filtered']]
fBarMat[fBarMat>0] = 1
adata = adata[fBarMat.sum(axis=1) > 0,:]

/tmp/ipykernel_4942/2543604733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fBarMat[fBarMat>0] = 1
/tmp/ipykernel_4942/2543604733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fBarMat[fBarMat>0] = 1


In [15]:
adata.shape

(325203, 6685)

In [16]:
adata.uns['feature_KO_barcode_names_filtered'] = [ x for x in adata.uns['feature_barcode_names_filtered'] if not (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET"))]


/home/eraslab1/miniconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [17]:
len(adata.uns['feature_KO_barcode_names_filtered'])

3204

In [18]:
adata.obs[adata.uns['feature_KO_barcode_names_filtered']].shape

(325203, 3204)

In [19]:
adata.obs[adata.uns['feature_KO_barcode_names_filtered']].sum(axis=1).value_counts()

1    284080
0     41123
dtype: int64

In [ ]:
adata.write('outputs/anndata/adata-hash-features_singlets_SingleKO_05242020_Filtered.h5ad')